<img src="./images/DLI_Header.png" width=400/>

# Fundamentals of Accelerated Data Science # 

## 02 - K-Means ##

**Table of Contents**
<br>
This notebook uses GPU-accelerated K-means to find the best locations for a fixed number of humanitarian supply airdrop depots. This notebook covers the below sections: 
1. [Environment](#Environment)
2. [Load Data](#Load-Data)
3. [K-Means Clustering](#K-Means-Clustering)
    * [Exercise #1 - Make Another `KMeans` Instance](#Exercise-#1---Make-Another-KMeans-Instance)
4. [Visualize the Clusters](#Visualize-the-Clusters)

## Environment ##
For the first time we import `cuml`, the RAPIDS GPU-accelerated library containing many common machine learning algorithms. We will be visualizing the results of your work in this notebook, so we also import `cuxfilter`.

In [1]:
# DO NOT CHANGE THIS CELL
import cudf
import cuml

import cuxfilter as cxf

## Load Data ##
For this notebook we load again the cleaned UK population data--in this case, we are not specifically looking at counties, so we omit that column and just keep the grid coordinate columns.

In [3]:
# DO NOT CHANGE THIS CELL
gdf = cudf.read_csv('./data/clean_uk_pop.csv', usecols=['easting', 'northing'])
print(gdf.dtypes)
gdf.shape

northing    float64
easting     float64
dtype: object


(58479894, 2)

In [4]:
gdf.head()

,northing,easting
0,515491.5313,430772.1875
1,503572.4688,434685.8750
2,517903.6563,432565.5313
3,517059.9063,427660.6250
4,509228.6875,425527.7813


<a name='#s2-3'></a>
## K-Means Clustering ##
The unsupervised K-means clustering algorithm will look for a fixed number *k* of centroids in the data and clusters each point with its closest centroid. K-means can be effective when the number of clusters *k* is known or has a good estimate (such as from a model of the underlying mechanics of a problem).

Assume that in addition to knowing the distribution of the population, which we do, we would like to estimate the best locations to build a fixed number of humanitarian supply depots from which we can perform airdrops and reach the population most efficiently. We can use K-means, setting *k* to the number of supply depots available and fitting on the locations of the population, to identify candidate locations.

GPU-accelerated K-means is just as easy as its CPU-only scikit-learn counterpart. In this series of exercises, you will use it to optimize the locations for 5 supply depots.

`cuml.KMeans()` will initialize a K-means instance. Use it now to initialize a K-means instance called `km`, passing the named argument `n_clusters` set equal to our desired number `5`. Use the `km.fit` method to fit `km` to the population's locations by passing it the population data. After fitting, add the cluster labels back to the `gdf` in a new column named `cluster`. Finally, you can use `km.cluster_centers_` to see where the algorithm created the 5 centroids.

Below we train a K-means clustering algorithm to find 5 clusters. 

In [5]:
# DO NOT CHANGE THIS CELL
# instantaite
km = cuml.KMeans(n_clusters=5)

# fit
km.fit(gdf)

# assign cluster as new column
gdf['cluster'] = km.labels_
km.cluster_centers_

,0,1
0,306647.898235,408370.452191
1,442109.465392,402673.747673
2,288997.149971,553805.430444
3,148770.463641,311786.805381
4,170553.110214,521605.459724


<a name='#s2-e1'></a>
## Exercise #1 - Make Another `KMeans` Instance ##

**Instructions**: <br>
* Modify the `<FIXME>` only and execute the below cell to instantiate a K-means instance with 6 clusters.
* Modify the `<FIXME>` only and execute the cell below to fit the data. 

In [6]:
km = cuml.KMeans(n_clusters=6)

In [7]:
km.fit(gdf)
gdf['cluster'] = km.labels_
km.cluster_centers_

,0,1,2
0,156942.821777,425534.234660,3.384935
1,541098.296590,415800.758548,1.000003
2,188442.901682,542164.754638,3.716473
3,404166.790803,395320.596086,0.869631
4,301678.636519,435787.150921,0.382444
5,143993.814440,284471.494937,2.970024


Click ... for solution. 

In [8]:
gdf.to_csv('gdf.csv')

In [2]:
gdf = cudf.read_csv('gdf.csv')

In [3]:
gdf['northing'] = (gdf['northing'] + 6600000)*1.2 - 1350000
gdf['easting'] = (gdf['easting'] - 500000)*1.2 - 100000

In [4]:
cxf_data = cxf.DataFrame.from_dataframe(gdf)

In [5]:
colorscheme = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

scatter_chart = cxf.charts.datashader.scatter(x='easting', y='northing', aggregate_col='cluster', aggregate_fn='mean', color_palette=colorscheme, tile_provider='CartoDark')

cluster_widget = cxf.charts.panel_widgets.multi_select('cluster')

dash = cxf_data.dashboard(charts=[scatter_chart], sidebar=[cluster_widget], theme=cxf.themes.dark, data_size_widget=True)

In [6]:
dash.app()

GridSpec(ncols=12, nrows=5)
    [0] GridSpec(height=800, ncols=12, nrows=5, sizing_mode='fixed', width=1200)
        [0] HoloViews(DynamicMap, height=800, sizing_mode='stretch_both', width=1200)
    [1] WidgetBox(styles={'border-color': '...})
        [0] Number(css_classes=['indicator'], default_color='#2B2B2B', font_size='18pt', format='{value:,}', name='Datapoints Selected', sizing_mode='stretch_width', title_size='14pt', value=58479894)
        [1] Progress(sizing_mode='stretch_width', styles={'--success-bg-color': '...}, value=100)
    [2] Column(min_height=500, sizing_mode='stretch_width')
        [0] MultiChoice(name='cluster', options=[1, 3, 4, 2, 0, 5], sizing_mode='scale_width', styles={'color': '#2B2B2B'}, stylesheets=['\n                .choic...])

<a id='#s2-4'></a>
## Visualize the Clusters ##
To help us understand where clusters are located, we make a visualization that separates them, using the same three steps as before.

Below we plot the clusters with cuxfilter. 

In [ ]:
# DO NOT CHANGE THIS CELL
# associate a data source with cuXfilter
cxf_data = cxf.DataFrame.from_dataframe(gdf)

# define charts
scatter_chart = cxf.charts.datashader.scatter(x='easting', y='northing')

# define widget using the `cluster` column for multiselect
# use the same technique to scale the scatterplot, then add a widget to let us select which cluster to look at
cluster_widget = cxf.charts.panel_widgets.multi_select('cluster')

In [ ]:
# create dashboard
dash = cxf_data.dashboard(charts=[scatter_chart],sidebar=[cluster_widget], theme=cxf.themes.dark, data_size_widget=True)

In [ ]:
dash.app()

In [31]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

**Well Done!** Let's move to the [next notebook](2-03_dbscan.ipynb). 

<img src="./images/DLI_Header.png" width=400/>